<a href="https://colab.research.google.com/github/jonatanbarkan/ACEDeep/blob/master/ADCEs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, TensorDataset
import tqdm
import pandas as pd

sns.set_style("whitegrid")


In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
d = 3
num_samples = 100000

In [ ]:
mu = [0]*d
cov = np.eye(d)

In [ ]:
kind = 'hex'
# kind = 'scatter'

In [ ]:
X = np.random.multivariate_normal(mu, cov, num_samples)
Y = np.random.multivariate_normal(mu, cov, num_samples)

In [ ]:
np.std(X, axis=0)

In [ ]:
thetaX = [0] * d
thetaX[2] = np.pi/1
rotX = Rotation.from_rotvec(thetaX)
X_rot = rotX.apply(X)

In [ ]:
np.mean(X_rot, axis=0)

In [ ]:
sns.jointplot(X, X_rot, kind=kind)

In [ ]:
thetaY = [0] * d
thetaY[0] = -np.pi / 2
thetaY[1] = -np.pi / 6
rotY = Rotation.from_rotvec(thetaY)
Y_rot = rotY.apply(Y)

In [ ]:
np.std(Y_rot, axis=0)

In [ ]:
sns.jointplot(Y, Y_rot, kind=kind)

In [ ]:
sns.jointplot(X, Y, kind=kind)
sns.jointplot(X_rot, Y_rot, kind=kind)

In [ ]:
## X, Y are num_samples samples of a multivariate gaussian with 0 mean and unit covariance

In [ ]:
x_tensor = torch.from_numpy(X_rot).float()
y_tensor = torch.from_numpy(Y_rot).float()

In [ ]:
dataset = TensorDataset(x_tensor, y_tensor)

In [ ]:
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len

In [ ]:
train_dataset, val_dataset = random_split(dataset, [train_len, val_len])

In [ ]:
class Model(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, lr=0.01, autoregularization=False, activation=None):
      super(Model, self).__init__()
      self.fc1 = nn.Linear(input_size, hidden_size)
      # nn.init.xavier_normal_(self.fc1.weight)
      self.bn1 = nn.BatchNorm1d(num_features=hidden_size)
      self.fc2 = nn.Linear(hidden_size, output_size)
      # nn.init.xavier_normal_(self.fc2.weight)
      self.bn2 = nn.BatchNorm1d(num_features=output_size, )
      self.autoregularization = autoregularization
      if self.autoregularization:
        self.fc2_inv = nn.Linear(output_size, hidden_size)
        self.bn2_inv = nn.BatchNorm1d(num_features=hidden_size)
        self.fc1_inv = nn.Linear(hidden_size, input_size)
        
      self.learning_rate = lr
      self._criterion = nn.MSELoss()
      self._optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
      self._reg = 0.01
      self.activation = activation
      
  def forward(self, x, activation=None):
      z = self.fc1(x)
      z = self.bn1(z)
      if self.activation is not None:
        z = self.activation(z)
      z = self.fc2(z)
      if self.autoregularization:
        x = self.fc2_inv(z)
        x = self.bn2_inv(x)
        if self.activation is not None:
          z = self.activation(z)
        x = self.fc1_inv(x)
      return z, x
    
  def get_criterion(self):
      return self._criterion

  def get_optimizer(self):
      return self._optimizer
  
  def get_reg(self):
      return self._reg

In [ ]:
def make_train_step(model):
    # Builds function that performs a step in the train loop
    loss_fn = model.get_criterion()
    optimizer = model.get_optimizer()
    autoregularization = model.autoregularization
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        yhat, x_hat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        if autoregularization:
          loss += model.get_reg() * loss_fn(x, x_hat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return train_step

In [ ]:
def make_eval_step(model):
    # Builds function that performs a step in the train loop
    loss_fn = model.get_criterion()
    def eval_step(x, y):
        # Sets model to TRAIN mode
        model.eval()
        # Makes predictions
        yhat, x_reg = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return eval_step

In [ ]:
def apply_transform(model, inp, zero_mean=True, unit_var=True):
  model.eval()
  out, reconstraction = model(inp)
  if zero_mean:
    out -= out.mean(axis=0)
  if unit_var:
    out = torch.div(out, out.std(axis=0))
  return out

In [ ]:
wanted_dim = 1
hidden_size = 3
max_epochs = 20
zero_mean=True
unit_var=True
lr = 0.01
act = None
autoregularization = False
batch_size = train_len

In [26]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size)

In [27]:
from sklearn.cross_decomposition import CCA
cca = CCA(n_components=1)
cca.fit(X_rot, Y_rot)
X_c, Y_c = cca.transform(X, Y)
print(X_c.mean(),Y_c.mean())
print(X_c.std(),Y_c.std())
print('score = ', np.mean(X_c * Y_c))

In [ ]:
x_model = Model(X_rot.shape[-1], hidden_size, wanted_dim, lr=lr, activation=act, autoregularization=autoregularization)
y_model = Model(Y_rot.shape[-1], hidden_size, wanted_dim, lr=lr, activation=act, autoregularization=autoregularization)

# x_model_knn = KNN(hidden_size)

losses_x = []
losses_y = []
expectations = []
val_expectations = []
steps = []
epochs = []
val_losses_x = []
val_losses_y = []

x_train_step = make_train_step(x_model)
y_train_step = make_train_step(y_model)

x_eval_step = make_eval_step(x_model)
y_eval_step = make_eval_step(y_model)
last_score = -np.infty

i = 0
ep = 0

for epoch in range(max_epochs):
  
  for x_batch, y_batch in train_loader:
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    transformed_y_batch = apply_transform(y_model, y_batch, zero_mean, unit_var)
    loss_x = x_train_step(x_batch, transformed_y_batch)
    losses_x.append(loss_x)

    transformed_x_batch = apply_transform(x_model, x_batch, zero_mean, unit_var)
    loss_y = y_train_step(y_batch, transformed_x_batch)
    losses_y.append(loss_y)
   
    transformed_y_batch = apply_transform(y_model, y_batch, zero_mean, unit_var)
    corr = torch.mul(transformed_x_batch, transformed_y_batch)
    expectations.append(corr.mean().item())

    steps.append(i)
    i += 1
    epochs.append(ep)
  val_loss_x = []
  val_loss_y = []
  val_expect = []
  U = []
  V = []
  with torch.no_grad(): 
    for x_val, y_val in val_loader:
      x_val = x_val.to(device)
      y_val = y_val.to(device)

      transformed_y_batch = apply_transform(y_model, y_val, zero_mean, unit_var)
      transformed_x_batch = apply_transform(x_model, x_val, zero_mean, unit_var)

      U.append(transformed_x_batch)
      V.append(transformed_y_batch)

      val_loss_x.append(x_eval_step(x_val, transformed_y_batch))
      val_loss_y.append(y_eval_step(y_val, transformed_x_batch))

  val_losses_x.append(np.mean(val_loss_x))
  val_losses_y.append(np.mean(val_loss_y))

  UV = torch.cat(U) * torch.cat(V)
  current_score = UV.mean(axis=0).item()
  val_expectations.append(UV.mean(axis=0).item())
  # val_expect_temp = torch.cat(val_expect).mean(axis=0).numpy()
  # val_expectations.append(torch.cat(val_expect).mean(axis=0).item())
  if last_score > current_score:
    break
  ep += 1
        
# print(x_model.state_dict())
g = sns.lineplot(range(len(val_expectations)), val_expectations)

In [35]:
dat = pd.DataFrame({'ace score': val_expectations, 'linear score': [np.mean(X_c * Y_c)] * len(val_expectations), 'epoch': range(len(val_expectations))})

In [36]:
gb = dat.groupby('epoch')

In [50]:
df_2 = gb.mean()
df_2.head(5)

,ace score,linear score
epoch,,
0,-0.001712,-0.007216
1,-0.001768,-0.007216
2,-0.001798,-0.007216
3,-0.001803,-0.007216
4,-0.001785,-0.007216


In [49]:
sns.lineplot('ace score', 'linear score')

ValueError: ignored

In [ ]:
losses = losses_x + losses_y
steps_col = steps + steps
epochs_col = epochs + epochs
it = len(epochs)
model_name = it * ['x model'] + it * ['y model']

In [ ]:
d = {'training loss': losses, 'model': model_name, 'steps': steps_col, 'epoch': epochs_col}
df = pd.DataFrame(d)

In [ ]:
sns.lineplot('steps', 'training loss', 'model', data=df)

In [ ]:
sns.lineplot(steps, expectations)

In [ ]:
sns.lineplot(range(len(val_losses_x)), val_losses_x)

In [ ]:
sns.lineplot(range(len(val_losses_y)), val_losses_y)

In [ ]:
transformed_y_batch.std()

In [ ]:
print(X_c.std(),Y_c.std())

In [ ]:
np.mean(X_c * Y_c)

In [ ]:
np.corrcoef(X_c, Y_c)